## Database de conversion unité/poids

L'objectif de ce notebook est de créer une database contenant les poids de certains aliments afin de permettre à notre utilisateur d'entrer ses aliments en unités plutôt qu'en poids. 
Par exemple, s'il réalise une bolognaise, il va entrer le nombre de tomates et non le poids de tomates dans sa recette. 

Pour cela, nous allons scrapper une page internet qui contient un tableau les poids de différents fruits et légumes. 

Nous allons nous servir de cette database comme point de départ et nous pourrons plus tard ajouter à notre database les aliments que nous jugeons pertinents (par exemple le poids d'un oeuf). 

In [5]:
import urllib
import bs4
import pandas
import numpy as np
from urllib import request
import requests

In [6]:
convert_page = requests.get("https://www.bloc-notes-culinaire.com/2019/09/poids-moyen-des-fruits-et-legumes.html#")

In [7]:
clean_convert_page = bs4.BeautifulSoup(convert_page.content, "lxml")

In [8]:
elem = clean_convert_page.findAll("table",{"class": "innerTable"})

On retraite complètement le résultat du scrapping pour garder uniquement le nom des légumes. 

In [20]:
legumes = []
for element in elem[0]: 
    word_element = "'%s'"%element
    legumes.append(word_element)
legumes
#print(legumes, len(legumes))
legumes = list(filter(("'\n'").__ne__, legumes))
legumes = list(filter(("'<tr><th>Variété</th><th>Poids moyen (kg)</th></tr>'").__ne__, legumes))

In [26]:
import re
data_convert = pandas.DataFrame(columns = ['Aliment','Poids'])
for element in legumes: 
    aliment = re.sub('<tr><td>|</td></tr>|</td><td>|0|1|2|3|4|5|6|7|8|9|,','',element)
    gramme = re.sub('[^0123456789,]', '',element)
    vect = [aliment,gramme]
    df = pandas.DataFrame([vect], columns = ['Aliment','Poids'])
    data_convert = data_convert.append(df)

display(data_convert.head())

,Aliment,Poids
0,'Ail tête',"0,080"
0,'Ail gousse',"0,007"
0,'Artichaut Camus',"0,350"
0,'Artichaut Castel',"0,500"
0,'Artichaut Poivrade',"0,060"


### On ajoute l'ingrédient Oeuf
On pourrait en faire de même pour de nombreux autres ingrédients pour encore plus affiner la qualité de notre application

In [24]:
vect_oeuf = ["'Oeuf'",'0,060']
df_oeuf = pandas.DataFrame([vect_oeuf], columns = ['Aliment','Poids'])
data_convert = data_convert.append(df_oeuf)

In [25]:
display(data_convert.tail())

,Aliment,Poids
0,'Pomme de terre M',"0,120"
0,'Pomme de terre P',"0,080"
0,'Tomate G',"0,200"
0,'Tomate P',"0,075"
0,'Oeuf',"0,060"


### On crée le fichier csv pour stocker cette database

In [20]:
import csv
data_convert.to_csv('data_convert.csv')